# Basic Convolutional Neural Networks using Tensorflow

These are my notes for making a basic CNN using the mnist digit recognizer.

First, import the data, which has already been preprocessed, from MNIST. Lets also import tensorflow. Our dataset will be 28 pixel by 28 pixel black and white images of digits.

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)
import tensorflow as tf

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting ./train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting ./train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./t10k-labels-idx1-ubyte.gz


In [2]:
mnist

Datasets(train=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x10880de48>, validation=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x10880df60>, test=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x118016550>)

Initialize parameters. Lets use a small learning rate of 0.00001, 10 epochs, batch size of 128, validation set size of 256, and 10 classes (one for each digit). For convolutional layers, lets use a convolutional filter size of 5x5, convolutional stride of 1, and 'same' padding. Let use max pooling with a 2x2 pooling size, a pooling stride of 2.  Finally, lets use a dropout probability of 0.75.

In [5]:
learning_rate = 0.00001 
epochs = 10 # Number of times you pass through the entire data sample.
batch_size = 128 # Number of samples propagated through the network at a time. Requires less memory than doing the whole network at the same time, and also trains much faster.
validation_set_size = 256
num_classes = 10  # (0-9 digits)

conv_filter_size = 5 # Use a 4x4 convolutional filter.
conv_filter_stride = 1
padding = 'SAME'

pooling_size = 2 # Take the max value of a 2x2 matrix.
pooling_stride = 2

keep_prob_value = 0.75  # Probability of keeping a unit during dropout.

Lets design our CNN as follows: input layer, convolutional layer 1, convolutional layer 2, regular layer 1, and output layer. Lets use the relu nonlinear activation function for convolutional layer 1, convolutional layer 2, and regular layer 1.

In [6]:
weights = {
    'wc1': tf.Variable(tf.random_normal([conv_filter_size, conv_filter_size, 1, 32])),
    'wc2': tf.Variable(tf.random_normal([conv_filter_size, conv_filter_size, 32, 64])),
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    'out': tf.Variable(tf.random_normal([1024, num_classes]))}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([num_classes]))}


def conv2d(x, weight, bias, conv_filter_stride, padding):
    conv = tf.nn.conv2d(x, weight, [1, conv_filter_stride, conv_filter_stride, 1], padding)
    conv = tf.nn.bias_add(conv, bias)
    return tf.nn.relu(conv)

def maxpool2d(x, pooling_size, pooling_stride, padding):
    return tf.nn.max_pool(x, [1, pooling_size, pooling_size, 1], [1, pooling_stride, pooling_stride, 1], padding)

def conv_net(x, weights, biases, keep_prob):
    # Convolutional layer 1 - 28*28*1 to 14*14*32
    conv1 = conv2d(x, weights['wc1'], biases['bc1'], conv_filter_stride, padding)
    conv1 = maxpool2d(conv1, pooling_size, pooling_stride, padding)

    # Convolutional layer 2 - 14*14*32 to 7*7*64
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'], conv_filter_stride, padding)
    conv2 = maxpool2d(conv2, pooling_size, pooling_stride, padding)

    # Fully connected layer - 7*7*64 to 1024
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, keep_prob)

    # Output Layer - class prediction - 1024 to 10
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

print("CNN design initialized.")

CNN design initialized.


Now, lets create some of the things we need to run our neural net.

In [7]:
# tf Graph input
x = tf.placeholder(tf.float32, [None, 28, 28, 1])
y = tf.placeholder(tf.float32, [None, num_classes])
keep_prob = tf.placeholder(tf.float32)

# Model
logits = conv_net(x, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

Finally, run the neural net! Hopefully it doesn't suck too much...

In [13]:
# Launch the graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for epoch in range(epochs):
        for batch in range(mnist.train.num_examples//batch_size):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict={x: batch_x, y: batch_y, keep_prob: 0.75})

            # Calculate batch loss and accuracy
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y, keep_prob: 0.75})
            valid_acc = sess.run(accuracy, feed_dict={
                x: mnist.validation.images[:validation_set_size],
                y: mnist.validation.labels[:validation_set_size],
                keep_prob: 1.})

            print('Epoch {:>2}, Batch {:>3} - Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(
                epoch + 1,
                batch + 1,
                loss,
                valid_acc))

    # Calculate Test Accuracy
    test_acc = sess.run(accuracy, feed_dict={
        x: mnist.test.images[:test_valid_size],
        y: mnist.test.labels[:test_valid_size],
        keep_prob: 1.})
    print('Testing Accuracy: {}'.format(test_acc))


Epoch  1, Batch   1 - Loss: 67377.7266 Validation Accuracy: 0.082031
Epoch  1, Batch   2 - Loss: 60716.5820 Validation Accuracy: 0.097656
Epoch  1, Batch   3 - Loss: 49919.8125 Validation Accuracy: 0.113281
Epoch  1, Batch   4 - Loss: 57885.5273 Validation Accuracy: 0.140625
Epoch  1, Batch   5 - Loss: 54149.2031 Validation Accuracy: 0.148438
Epoch  1, Batch   6 - Loss: 44317.6914 Validation Accuracy: 0.148438
Epoch  1, Batch   7 - Loss: 41870.4922 Validation Accuracy: 0.183594
Epoch  1, Batch   8 - Loss: 45318.2031 Validation Accuracy: 0.167969
Epoch  1, Batch   9 - Loss: 42233.7148 Validation Accuracy: 0.167969
Epoch  1, Batch  10 - Loss: 46460.7656 Validation Accuracy: 0.164062
Epoch  1, Batch  11 - Loss: 41880.0352 Validation Accuracy: 0.195312
Epoch  1, Batch  12 - Loss: 43589.4453 Validation Accuracy: 0.203125
Epoch  1, Batch  13 - Loss: 44426.3359 Validation Accuracy: 0.203125
Epoch  1, Batch  14 - Loss: 38501.7227 Validation Accuracy: 0.191406
Epoch  1, Batch  15 - Loss: 37110.

Epoch  1, Batch 120 - Loss: 11945.5176 Validation Accuracy: 0.625000
Epoch  1, Batch 121 - Loss: 11110.9189 Validation Accuracy: 0.628906
Epoch  1, Batch 122 - Loss: 10286.7734 Validation Accuracy: 0.621094
Epoch  1, Batch 123 - Loss: 11292.5547 Validation Accuracy: 0.628906
Epoch  1, Batch 124 - Loss: 11744.3350 Validation Accuracy: 0.628906
Epoch  1, Batch 125 - Loss: 12417.9824 Validation Accuracy: 0.644531
Epoch  1, Batch 126 - Loss: 10278.1016 Validation Accuracy: 0.640625
Epoch  1, Batch 127 - Loss:  9878.3926 Validation Accuracy: 0.656250
Epoch  1, Batch 128 - Loss: 13725.9863 Validation Accuracy: 0.656250
Epoch  1, Batch 129 - Loss:  8910.6191 Validation Accuracy: 0.652344
Epoch  1, Batch 130 - Loss: 11989.2188 Validation Accuracy: 0.664062
Epoch  1, Batch 131 - Loss: 10144.7129 Validation Accuracy: 0.667969
Epoch  1, Batch 132 - Loss:  9692.2920 Validation Accuracy: 0.660156
Epoch  1, Batch 133 - Loss: 11137.6289 Validation Accuracy: 0.656250
Epoch  1, Batch 134 - Loss: 12763.

Epoch  1, Batch 239 - Loss:  8440.2998 Validation Accuracy: 0.753906
Epoch  1, Batch 240 - Loss:  5569.3765 Validation Accuracy: 0.757812
Epoch  1, Batch 241 - Loss:  6429.1523 Validation Accuracy: 0.757812
Epoch  1, Batch 242 - Loss:  6945.4795 Validation Accuracy: 0.765625
Epoch  1, Batch 243 - Loss:  6012.9351 Validation Accuracy: 0.761719
Epoch  1, Batch 244 - Loss:  6369.4580 Validation Accuracy: 0.769531
Epoch  1, Batch 245 - Loss:  4822.4121 Validation Accuracy: 0.773438
Epoch  1, Batch 246 - Loss:  7189.6104 Validation Accuracy: 0.765625
Epoch  1, Batch 247 - Loss:  6521.1113 Validation Accuracy: 0.761719
Epoch  1, Batch 248 - Loss:  5958.1455 Validation Accuracy: 0.765625
Epoch  1, Batch 249 - Loss:  6475.0186 Validation Accuracy: 0.769531
Epoch  1, Batch 250 - Loss:  6277.2993 Validation Accuracy: 0.765625
Epoch  1, Batch 251 - Loss:  6325.1221 Validation Accuracy: 0.761719
Epoch  1, Batch 252 - Loss:  6196.2031 Validation Accuracy: 0.757812
Epoch  1, Batch 253 - Loss:  5256.

Epoch  1, Batch 358 - Loss:  4429.1440 Validation Accuracy: 0.820312
Epoch  1, Batch 359 - Loss:  4085.9180 Validation Accuracy: 0.816406
Epoch  1, Batch 360 - Loss:  5414.0474 Validation Accuracy: 0.816406
Epoch  1, Batch 361 - Loss:  4330.7407 Validation Accuracy: 0.816406
Epoch  1, Batch 362 - Loss:  5353.8198 Validation Accuracy: 0.832031
Epoch  1, Batch 363 - Loss:  4751.0664 Validation Accuracy: 0.824219
Epoch  1, Batch 364 - Loss:  4257.9561 Validation Accuracy: 0.828125
Epoch  1, Batch 365 - Loss:  5488.0029 Validation Accuracy: 0.828125
Epoch  1, Batch 366 - Loss:  4300.6260 Validation Accuracy: 0.824219
Epoch  1, Batch 367 - Loss:  6096.9922 Validation Accuracy: 0.824219
Epoch  1, Batch 368 - Loss:  3128.4941 Validation Accuracy: 0.828125
Epoch  1, Batch 369 - Loss:  4947.4893 Validation Accuracy: 0.828125
Epoch  1, Batch 370 - Loss:  4264.2651 Validation Accuracy: 0.828125
Epoch  1, Batch 371 - Loss:  3487.5215 Validation Accuracy: 0.832031
Epoch  1, Batch 372 - Loss:  5137.

KeyboardInterrupt: 